In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, not_
from dotenv import load_dotenv
import os
import sys
import requests
from typing import Any
from datetime import datetime
from models import Player, GameStats, KtcValue

In [7]:
def fetch(url) -> dict[str, Any]:
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.json()
    raise ValueError

def get_rosters(league_id) -> dict[str, Any]:
    url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
    return fetch(url)

def fetch_all_players() -> dict[str, Any]:
    url = "https://api.sleeper.app/v1/players/nfl"
    return fetch(url)

In [8]:
all_players = fetch_all_players()

In [9]:
load_dotenv()
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=False)

Session = sessionmaker(bind=engine)
session = Session()

In [38]:
id = '8040'
print(f"{all_players[id]['full_name']}: {all_players[id]['position']}")

Quinn Nordin: K


In [30]:
missing_players = []

for player_id in missing_players:
    player_id = str(player_id)

    player_data = all_players[player_id]
    player = Player(

    )
    player.birth_date = datetime.strptime(player_data.get('birth_date'), '%Y-%m-%d')
    player.display_name = str(player_data.get('full_name'))
    player.weight = int(player_data.get('weight'))
    player.height = int(player_data.get('height'))
    player.position = str(player_data.get('position'))
    player.rookie_year = int(player_data['metadata']['rookie_year'])
    player.college = str(player_data.get('college'))
    player.team_abbr = str(player_data.get('team'))
    player.sleeper_id = str(player_data.get('player_id'))
    player.rotoworld_id = str(player_data.get('rotoworld_id', None))
    player.rotowire_id = str(player_data.get('rotowire_id', None))
    player.yahoo_id = str(player_data.get('yahoo_id', None))
    player.espn_id = str(player_data.get('espn_id', None))
    player.swish_id = str(player_data.get('swish_id', None))
    session.add(player)
session.commit()



In [29]:
missing_gsis = []

for player_data in all_players.values():
    gsis_id = player_data.get('gsis_id')
    if gsis_id:
        player = session.query(Player).filter_by(gsis_id=gsis_id).first()
        if player:
            player.sleeper_id = str(player_data.get('player_id'))
            player.rotoworld_id = str(player_data.get('rotoworld_id', None))
            player.rotowire_id = str(player_data.get('rotowire_id', None))
            player.yahoo_id = str(player_data.get('yahoo_id', None))
            player.espn_id = str(player_data.get('espn_id', None))
            player.swish_id = str(player_data.get('swish_id', None))
            session.add(player)
        else:
            missing_players.append(player_data['full_name'])
            missing_gsis.append(gsis_id)
session.commit()


In [ ]:
def update_player_by_name(session, name, sleeper_id):
    players = session.query(Player).filter(Player.display_name.ilike(f"%{name}%")).all()

    if not players:
        return False, "Not Found"
    
    for i, player in enumerate(players):
        ktc_count = len(player.ktc_values) if player.ktc_values else 0
        print(f"{i+1}: {player.display_name}, POS: {player.position}, Sleeper ID: {player.sleeper_id}, KTC Connections: {ktc_count} -> {sleeper_id}")

    if len(players) > 1:
        print("Please review the listed players to select the correct one to update:")
        choice = int(input("Enter the number of the player to update: ")) - 1

        clear_output(wait=True)
        player_to_update = players[choice]


        if choice == -1:
            print('Storing for later')
            return False, "Not Found"
    else:
        player_to_update = players[0]
        
    player_to_update.sleeper_id = sleeper_id
    session.commit()
    print("Player Successfully Updated.")
    return True, "Successful"